In [ ]:
import os
import json
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pysam
from collections import Counter

%matplotlib inline
plt.rcParams['figure.figsize'] = [16.5, 5]
plt.rcParams['font.size'] = 12

Define a function that extracts repeat allele sequences from each read. This function will return two lists containing sequences of each allele.

In [ ]:
def get_seqs(path, trid):
    bam = pysam.AlignmentFile(trgt_bam_path)
    seqs_by_allele = ([], [])
    for read in bam.fetch(until_eof=True):
        allele = read.get_tag("AL")
        flank_lens = read.get_tag("FL")
        lf, rf = list(flank_lens)
        seq = read.query_sequence[lf:-rf]
        seqs_by_allele[allele].append(seq)
    return seqs_by_allele

Specify a path to the BAM file we want to analyze. This must be the spanning reads BAM generated by TRGT.

In [ ]:
trgt_bam_path = "input/trgt.bam"

Let's use our function to extract repeat alleles.

In [ ]:
seqs_by_allele = get_seqs(trgt_bam_path, "FMR1")

Here are the sequences of the first allele:

In [ ]:
seqs_by_allele[0]

And the sequences of the second allele:

In [ ]:
seqs_by_allele[1]

Plot a histogram depicting the counts of reads with different length (stratified by allele)

In [ ]:
def bar_plot(values, ax, label):
    counter = Counter(values)
    xs = counter.keys()
    ys = [counter[x] for x in xs]
    ax.bar(xs, ys, label=label)


fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 3))
bar_plot([len(seq) for seq in seqs_by_allele[0]], ax, "allele 1")
bar_plot([len(seq) for seq in seqs_by_allele[1]], ax, "allele 2")
ax.set_xlabel("Repeat size (bps)")
ax.set_ylabel("Read count")
plt.legend();